# Senata Elections, 1976-2020
This Notebook computes the tables required for the Senate Elections example dashboard.

Step 1: Import the libraries

In [101]:
import pandas as pd
from galyleo.galyleo_table import GalyleoTable
from galyleo.galyleo_jupyterlab_client import GalyleoClient

Use Pandas to read the libraries, and view the result

In [199]:
elections = pd.read_csv('1976-2020-senate.csv')


Cut out irrelevant columns, compute the percentage, and trim out candidates who got < 5% of the vote

In [165]:
elections_trimmed = elections[['year', 'state', 'special', 'candidate', 'party_simplified', 'candidatevotes', 'totalvotes']]
elections_two = elections_trimmed.assign(percentage=pd.Series(elections_trimmed['candidatevotes']/elections_trimmed['totalvotes'] * 100))
elections_three = elections_two[elections_two['percentage'] > 5]

A convenience method to send a dataframe to a dashboard, with table name name, using GalyleoClient client

In [166]:
def send_dataframe_to_dashboard_table(dataframe, table_name, client):
    table = GalyleoTable(table_name)
    table.load_from_dataframe(dataframe)
    client.send_data_to_dashboard(table)

In [167]:
client = GalyleoClient()
send_dataframe_to_dashboard_table(elections_three, 'senate_vote', client)

A convenience  method to take a dataframe, add columns for the values in split_column_name, and populate those columns with the appropriate value from values_column_name.  The use case for this in this Notebook is to take the percentage column and turn it into four columns: DEMOCRAT, REPUBLICAN, LIBERTARIAN, AND OTHER.
Parameters: 
  - dataframe: the source dataframe
  - index_columns: the columns to use as indexes -- common across all the daughter tables
  - split_column_name: the column containing the names of the new columns
  - values_column_name: the column containing the values for the new columns

In [187]:
def split_column(dataframe, index_columns, split_column_name, values_column_name):
    keep_columns = index_columns + [values_column_name]
    table_names = set([v for v in dataframe[split_column_name]])
    tables = []
    for name in table_names:
        table = dataframe[dataframe[split_column_name] == name][keep_columns]
        tables.append(table.rename(columns = {values_column_name: name}).set_index(index_columns))
    result = tables[0]
    for table in tables[1:]:
        result = result.join(table, how = 'outer')
    return result.reset_index().fillna(0)

In [188]:
by_party = split_column(elections_three, ['year', 'state', 'special'], 'party_simplified', 'percentage')

Compute the margin for a row.  This should be +5 to + 10 for a Democrat victory, 0 for OTHER or LIBERTARIAN, and -5 to -10 for a REPUBLICAN victory.  5 is a DEMOCRAT squeaker, 10 a landslide, -5 a REPUBLICAN squeaker, -10 a landslide

In [189]:
import math

def compute_margin(row):
    keys = ['DEMOCRAT', 'REPUBLICAN', 'LIBERTARIAN', 'OTHER']
    parties = [(key, row[key]) for key in keys]
    parties.sort(key=lambda party: party[1], reverse=True)
    if (parties[0][0] in {'LIBERTARIAN', 'OTHER'}): return 0
    margin = min(math.floor((parties[0][1] - parties[1][1])/2), 5)
    return margin + 5 if  parties[0][0] == 'DEMOCRAT' else -(5 + margin)

In [190]:
by_party['margin'] = by_party.apply(lambda row: compute_margin(row), axis = 1)

Send the vote history table to the dashboard

In [191]:
send_dataframe_to_dashboard_table(by_party[['year', 'state', 'special', 'REPUBLICAN', 'DEMOCRAT', 'LIBERTARIAN', 'OTHER']], 'history', client)

Send the margin table to the dashboard

In [192]:
send_dataframe_to_dashboard_table(by_party[['year', 'state', 'special', 'margin']], 'margin', client)

Read the Senate control table (how many seats before/after each election) and send to dashboard

In [193]:
control = pd.read_csv('senate-control.csv')
send_dataframe_to_dashboard_table(control, 'senate-control', client)